### Getting started

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import maria 
from maria import models

In order to construct a model, we need some ingredients.

The first ingredient is the array, which entirely defines the telescope. The Array object contains all the information about detector offsets, baselines, frequencies, and more, as well as apertures and observational constraints on the instrument.

We can load one of `maria`'s predefined arrays ([MUSTANG-2](https://greenbankobservatory.org/science/gbt-observers/mustang-2/)) 

In [ ]:
mustang_2 = maria.get_array("MUSTANG-2")
mustang_2.description

and plot the detector offsets:

In [ ]:


plt.figure(figsize=(4, 4))

for band in mustang_2.ubands:

    band_mask = mustang_2.band == band

    plt.scatter(60 * np.degrees(mustang_2.offset_x[band_mask]), 
                60 * np.degrees(mustang_2.offset_y[band_mask]),
                label=f'{1e-9 * band}', lw=5e-1)

plt.xlabel(r'$\theta_x$ offset (arcminutes)')
plt.ylabel(r'$\theta_y$ offset (arcminutes)')
plt.legend()

The second ingredient is the pointing, which determines where and how the telescope moves, and how fast it samples. 

We can loading a default scan with a two-degree radius from `maria`:

In [ ]:
daisy_scan = maria.get_pointing("DAISY_2deg_4ra_10dec_600s")
daisy_scan.description

In [ ]:
plt.figure(figsize=(4,4))

plt.plot(np.degrees(daisy_scan.ra), 
         np.degrees(daisy_scan.dec),
         lw=5e-1)

plt.xlabel(r'RA (degrees)')
plt.ylabel(r'Dec. (degrees)')

The last ingredient is the site, which determines the translation between local coordinates and sky coordinates. It also determines the weather parameters that go into the atmospheric simulation. 

Loading the Green Bank Telescope site from `maria`:

In [ ]:
green_bank = maria.get_site("GBT")
green_bank.description

We can combine these objects by passing them to one of `maria`'s models. An efficient, lightweight model is the linear angular model:

In [ ]:
lam = maria.models.LinearAngularModel(array=mustang_2, pointing=daisy_scan, site=green_bank)

This also computes things which depend on multiple inputs ()



In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,4))

axes[0].plot(np.degrees(lam.pointing.ra), 
             np.degrees(lam.pointing.dec),
             lw=5e-1)

axes[0].set_xlabel(r'RA (degrees)')
axes[0].set_ylabel(r'Dec. (degrees)')

axes[1].plot(np.degrees(lam.pointing.az), 
             np.degrees(lam.pointing.el),
             lw=5e-1)

axes[1].set_xlabel(r'azimuth (degrees)')
axes[1].set_ylabel(r'elevation (degrees)')

In [ ]:

lam.simulate_temperature(units='K_RJ')
print(lam.temperature.shape)

The simulated time-ordered data is stored `lam.temperature`, which has shape `(n_freqs, n_dets, n_samples)`. Plotting the output as fluctuations:

In [ ]:
for uib, uband in enumerate(np.unique(lam.array.band)):

    band_mask = lam.array.band == uband
    band_mean = lam.temperature[band_mask].mean(axis=0)
    plt.plot(lam.pointing.unix, band_mean, label=f'{1e-9*uband:.00f} GHz mean')

plt.xlabel('time (seconds)')
plt.ylabel(r'temperature ($K_{RJ}$)')
plt.legend()